In [1]:
import cv2
import numpy as np 
import pytesseract
import imutils


img=cv2.imread(r"C:\Users\korkm\Downloads\9.1 licence_plate.jpg.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
filtered=cv2.bilateralFilter(gray,7,250,250)# köşeleri yumuşatma
edged=cv2.Canny(filtered,30,200)
edged1=cv2.Canny(gray,30,200)# yumuşatma olmadan köşeleri bulma denendi



contours=cv2.findContours(edged,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE,)
cnts=imutils.grab_contours(contours)
cnts=sorted(cnts,key=cv2.contourArea,reverse=True)[:10]# köşeleri sıraladık--- alana göre --ters çevirerek
screen=None # kapalı şekil bulup bulamadığımızı kontrol etmek adına böyle bir değişken oluştu

# köşelerini bulduğumuz şekilden kapalı bir plaka şeklinde geometrik şekil alacağız

for c in cnts:
    # bozuk şekillerin konturlarını yakalamak adına deneysel bir formül kullanıcaz
    epsilon=0.018*cv2.arcLength(c,True)
    approx =cv2.approxPolyDP(c,epsilon,True)
    if len(approx)==4: # eğer dört köşe var ise yani aprroxda 4 değer saklı ise  muhtemel plaka alanını screen ile aldık
        screen=approx
        break

mask=np.zeros(gray.shape,np.uint8)
new_img=cv2.drawContours(mask,[screen],0,(255,255,255),-1)# screen bölgesindeki konturları aldı
new_img=cv2.bitwise_and(img,img,mask=mask)#################################
###########################################################################


(x,y)=np.where(mask==255)# eğer beyaz ise konumları (x,y) ye at dedik 
(topx,topy)=(np.min(x),np.min(y))# en üst noktları minimum noktaları saptayarak sol üstü saptadık 
(bottomx,bottomy)=(np.max(x),np.max(y))
cropped=gray[topx:bottomx+1,topy:bottomy+1]


text=pytesseract.image_to_string(cropped,lang="eng")
print("detected text:",text)


cv2.imshow("cropped",cropped)
cv2.imshow("img",img)
cv2.imshow("edged",edged)
cv2.imshow("mask",mask)
cv2.imshow("masknew",new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

detected text: 300 SL 1957.

